In [1]:
import json
import re
import torch
import pandas as pd
import numpy as np
from datasets import load_metric
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# Model initialization
def initialize_model():
    model_name = "microsoft/Phi-3-mini-4k-instruct"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    
    # Initialize with 4-bit quantization for better memory efficiency
    quantization_config = BitsAndBytesConfig(load_in_4bit=True)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.float16,
        device_map="auto",
        quantization_config=quantization_config
    )
    return model, tokenizer

# Data loading
def load_finqa_dataset(path):
    with open(path, "r") as f:
        return json.load(f)

def preprocess_example(example, tokenizer):
    """Prepare structured input for Phi-3"""
    question = example["qa"].get("question", "No question available.")
    table = example.get("table", [])
    table_str = "\n".join([" | ".join(row) for row in table])
    
    pre_text = " ".join(example.get("pre_text", []))
    post_text = " ".join(example.get("post_text", []))
    
    gold_inds = example["qa"].get("gold_inds", {})
    relevant_info = "\n".join(gold_inds.values())
    program = example["qa"].get("program", "")
    
    input_text = (
        "You are a financial calculator. Follow these steps:\n"
        "1. Read the question carefully\n"
        "2. Look at the relevant information and table data\n"
        "3. Follow the mathematical operation exactly\n"
        "4. Return ONLY the final numerical answer with no text\n\n"
        f"Relevant Information:\n{relevant_info}\n\n"
        f"Table Data:\n{table_str}\n\n"
        f"Question: {question}\n"
        f"Mathematical Operation: {program}\n"
        "Final Answer (number only): "
    )

    return tokenizer(
        input_text,
        truncation=True,
        padding="max_length",
        max_length=2048,
        return_tensors="pt"
    )

def clean_answer(text):
    """Extract and format numerical answer"""
    if ':' in text:
        text = text.split(':')[-1]
    
    # Handle percentages first
    percent_match = re.search(r'[-+]?\d*\.?\d+\s*%?', text)
    if percent_match:
        number = float(percent_match.group(0).replace('%', '').strip())
        # If the number is small (likely decimal), convert to percentage
        if number < 1:
            number *= 100
        # Round to one decimal place and add % symbol
        return f"{round(number, 1)}%"
    
    # Handle regular numbers
    decimal_match = re.search(r'[-+]?\d*\.?\d+', text)
    if decimal_match:
        number = float(decimal_match.group(0))
        # If it's close to an integer, round it
        if abs(round(number) - number) < 0.01:
            return str(round(number))
        # Otherwise, round to one decimal place
        return str(round(number, 1))
    
    # Handle yes/no answers
    text = text.lower().strip()
    if 'yes' in text or 'true' in text:
        return 'yes'
    if 'no' in text or 'false' in text:
        return 'no'
    
    return text.strip()

def generate_answer(example, model, tokenizer):
    """Generate answer using Phi-3"""
    inputs = preprocess_example(example, tokenizer)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=10,
            do_sample=False,
            num_beams=5,
            temperature=0.1,
            top_p=0.95,
            early_stopping=True,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id
        )

    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    return clean_answer(generated_text)

def evaluate_model(test_data, model, tokenizer, num_samples=10):
    """Evaluate model performance"""
    metric = load_metric("squad")
    predictions = []
    references = []
    
    torch.cuda.empty_cache()
    
    for i, example in enumerate(test_data[:num_samples]):
        try:
            pred_text = generate_answer(example, model, tokenizer)
            true_text = example["qa"]["answer"]
            
            predictions.append({"id": str(i), "prediction_text": pred_text})
            references.append({"id": str(i), "answers": {"text": [true_text], "answer_start": [0]}})
            
            em = 1 if pred_text.strip() == true_text.strip() else 0
            f1 = metric.compute(
                predictions=[{"id": str(i), "prediction_text": pred_text}],
                references=[{"id": str(i), "answers": {"text": [true_text], "answer_start": [0]}}]
            )["f1"]
            
            print(f"\n🔹 Example {i+1}")
            print(f"❓ Question: {example['qa']['question']}")
            print(f"✅ Ground Truth: {true_text}")
            print(f"🤖 Prediction: {pred_text}")
            print(f"📊 Metrics - Exact Match: {em}, F1: {f1:.2f}")
            
        except Exception as e:
            print(f"Error processing example {i}: {str(e)}")
            continue
    
    results = metric.compute(predictions=predictions, references=references)
    print("\n📊 Overall Results:", results)
    return results

def main():
    # Initialize model and tokenizer
    model, tokenizer = initialize_model()
    print("Model initialized successfully!")
    
    # Load datasets
    test_data = load_finqa_dataset("/cs/student/projects2/aisd/2024/giliev/FinQA/dataset/test.json")
    print("Dataset loaded successfully!")
    
    # Evaluate model
    results = evaluate_model(test_data, model, tokenizer, num_samples=10)
    
    # Save results
    with open("evaluation_results.json", "w") as f:
        json.dump(results, f, indent=4)

if __name__ == "__main__":
    main()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Model initialized successfully!
Dataset loaded successfully!


/tmp/ipykernel_148719/2634805306.py:120: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("squad")
/cs/student/projects2/aisd/2024/giliev/miniconda3/envs/finqaEnv/lib/python3.9/site-packages/datasets/load.py:759: FutureWarning: The repository for squad contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/squad/squad.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/cs/student/projects2/aisd/2024/giliev/miniconda3/envs/finqaEnv/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:629: UserWarning

Error processing example 0: CUDA out of memory. Tried to allocate 640.00 MiB. GPU 0 has a total capacity of 23.58 GiB of which 10.12 MiB is free. Process 145777 has 12.70 GiB memory in use. Including non-PyTorch memory, this process has 10.78 GiB memory in use. Of the allocated memory 9.19 GiB is allocated by PyTorch, and 1.29 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)


/cs/student/projects2/aisd/2024/giliev/miniconda3/envs/finqaEnv/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:629: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/cs/student/projects2/aisd/2024/giliev/miniconda3/envs/finqaEnv/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:634: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Error processing example 1: CUDA out of memory. Tried to allocate 640.00 MiB. GPU 0 has a total capacity of 23.58 GiB of which 50.12 MiB is free. Process 145777 has 12.70 GiB memory in use. Including non-PyTorch memory, this process has 10.74 GiB memory in use. Of the allocated memory 9.19 GiB is allocated by PyTorch, and 1.25 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)
Error processing example 2: CUDA out of memory. Tried to allocate 640.00 MiB. GPU 0 has a total capacity of 23.58 GiB of which 50.12 MiB is free. Process 145777 has 12.70 GiB memory in use. Including non-PyTorch memory, this process has 10.74 GiB memory in use. Of the allocated memory 9.19 GiB is allocated by PyTorch, and 1.25 GiB is reserved by PyTorch but unallocated. If reserved

ZeroDivisionError: float division by zero